In [ ]:
import numpy as np
import pandas as pd
from datasets import load_dataset
# from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
from sentence_transformers import SentenceTransformer

### Talking points

- jump from previous stuff (not related to what we've learned so far)
- just soemthing cool
- think of it as "what else can we do with text and deep learning"

# RAG

- embeddings documents
- vector search
- contextual summarization

## Step 1: Pulling a dataset

In [ ]:
# Load the dataset
ds = load_dataset("SetFit/20_newsgroups")
ds


In [ ]:
# # Subsample targets which start with talk.politics
# df_train = df_train[df_train['label_text'].str.startswith('talk.politics')] # optional; sentence tf is very fast
# df_query = df_query[df_query['label_text'].str.startswith('talk.politics')] # optional; sentence tf is very fast

# df_train.shape, df_query.shape


# Embedding
Lets use a transformer model to create embedding representing the document.

In [ ]:
# Load the embedding model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Vector Store

(not necessary)

In [ ]:
import faiss 

# Lets try it out on some document

In [ ]:
i = 100
doc = df_query.iloc[i]
print(doc.text)

In [ ]:
## Create a query vector

In [ ]:
# Lets find the relevant indices using faiss

In [ ]:
# Seems good?

# Paraphrasing responses together

### How to use a local LLM?

We will use a lightweight qwen model -- https://ollama.com/library/qwen

In [ ]:
from ollama import chat, generate
from ollama import ChatResponse

response: ChatResponse = chat(model='qwen:0.5b', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])

response

In [ ]:
generate(model='qwen:0.5b', prompt="Why is the sky blue?")

In [ ]:
# Lets create a prompt

context = "\n\n".join(similar_articles)
prompt = f"""You are a helpful assistant. For a given user question, summarize or paraphrase the key points that are relevant to this question.
Note that the context is not a direct response to answer user question. Rather, we want to provide the user with a summary of what other people are asking on the topic. 
Your summary should not be too short. At least 4-8 sentences. In english.

User Question: 
--------------
```
{doc.text}
```

Context:
--------
{context}

Summary:"""

print(prompt)

In [ ]:
print(generate(model='qwen:0.5b', prompt=prompt).response)

## Fallback

In [ ]:
from transformers import pipeline

# Load summarization pipeline
summarizer = pipeline("summarization", "facebook/bart-large-cnn")

# Summarize each article
for i, article in enumerate(similar_articles, 1):
    summary = summarizer(article, max_length=100, min_length=30, do_sample=False)[0]['summary_text']
    print(f"\n--- Summary of Article {i} ---\n")
    print(summary)


In [ ]:
### Not exactly the same (dont expect this model to do what qwen can do)